In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plats = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/resultados_all_writtenReps_plats.csv"
jap = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/resultados_all_writtenReps_jap.csv"
gold = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/matches_gold.csv"


In [ ]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Leer los archivos CSV
plats_df = pd.read_csv(plats, encoding='utf-8')
jap_df = pd.read_csv(jap, encoding='utf-8')
gold_standard_df = pd.read_csv(gold, encoding='utf-8')  # Cargar el gold-standard

# Concatenar 'prefLabel' y 'writtenRep' de ambos archivos en una sola cadena para cada fila
plats_lines = plats_df['prefLabel'] + ". " + plats_df['writtenRep']
jap_lines = jap_df['prefLabel'] + ". " + jap_df['writtenRep']

# Limpiar y eliminar filas vacías
plats_lines = plats_lines.dropna().str.strip().tolist()
jap_lines = jap_lines.dropna().str.strip().tolist()

# Obtener los embeddings de las líneas de ambos archivos
plats_embeddings = model.encode(plats_lines, convert_to_tensor=True)
jap_embeddings = model.encode(jap_lines, convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Diccionario para convertir IDs a índices
plats_ids = plats_df['concepto'].tolist()
jap_ids = jap_df['concepto'].tolist()

# Leer los pares correctos del gold-standard
gold_standard_matches = set(zip(gold_standard_df['Plats_a_la_carta_ID'], gold_standard_df['Gastronomia_japonesa_ID']))

# Convertir el gold-standard a índices de filas y columnas de las matrices
gold_standard_indices = []
for plats_id, jap_id in gold_standard_matches:
    if plats_id in plats_ids and jap_id in jap_ids:
        plats_idx = plats_ids.index(plats_id)
        jap_idx = jap_ids.index(jap_id)
        gold_standard_indices.append((plats_idx, jap_idx))

# Contar cuántos pares del gold-standard superan el umbral de similitud de 0.75
umbral = 0.75
matches_above_threshold = 0

for plats_idx, jap_idx in gold_standard_indices:
    similarity = cosine_similarities[plats_idx, jap_idx].item()
    if similarity > umbral:
        matches_above_threshold += 1
        print(f"Gold-standard match: Plats ID {plats_ids[plats_idx]} - Jap ID {jap_ids[jap_idx]} con similitud {similarity:.4f}")

print(f"Número de matches del gold-standard que superan una similitud de {umbral}: {matches_above_threshold}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Gold-standard match: Plats ID plats_a_la_carta_C6528 - Jap ID gastronomia_japonesa_C10 con similitud 0.8657
Gold-standard match: Plats ID plats_a_la_carta_C6952 - Jap ID gastronomia_japonesa_C15 con similitud 0.7720
Número de matches del gold-standard que superan una similitud de 0.75: 2


In [ ]:
# Aplicar una máscara para encontrar todas las similitudes mayores al umbral
mask = cosine_similarities > umbral

# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Contador de matches totales que superan el umbral
total_matches_above_threshold = len(indices)

# Imprimir los pares que superan el umbral
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de 'plats' es similar a la línea {j} de 'jap' con similitud {similarity:.4f}")

# Mostrar el número total de matches que superan el umbral
print(f"Número total de matches que superan una similitud de {umbral}: {total_matches_above_threshold}")


Línea 982 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7549
Línea 1300 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7750
Línea 1386 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7509
Línea 4495 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7573
Línea 4582 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7975
Línea 4707 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7684
Línea 6143 de 'plats' es similar a la línea 1 de 'jap' con similitud 0.8657
Línea 6614 de 'plats' es similar a la línea 6 de 'jap' con similitud 0.7720
Número total de matches que superan una similitud de 0.75: 8


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Leer los archivos CSV
plats_df = pd.read_csv(plats, encoding='utf-8')
jap_df = pd.read_csv(jap, encoding='utf-8')
gold_standard_df = pd.read_csv(gold, encoding='utf-8')  # Cargar el gold-standard

# Utilizar solo 'writtenRep' de ambos archivos
plats_lines = plats_df['writtenRep'].dropna().str.strip().tolist()
jap_lines = jap_df['writtenRep'].dropna().str.strip().tolist()

# Obtener los embeddings de las líneas de 'writtenRep' de ambos archivos
plats_embeddings = model.encode(plats_lines, convert_to_tensor=True)
jap_embeddings = model.encode(jap_lines, convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Diccionario para convertir IDs a índices
plats_ids = plats_df['concepto'].tolist()
jap_ids = jap_df['concepto'].tolist()

# Leer los pares correctos del gold-standard
gold_standard_matches = set(zip(gold_standard_df['Plats_a_la_carta_ID'], gold_standard_df['Gastronomia_japonesa_ID']))

# Convertir el gold-standard a índices de filas y columnas de las matrices
gold_standard_indices = []
for plats_id, jap_id in gold_standard_matches:
    if plats_id in plats_ids and jap_id in jap_ids:
        plats_idx = plats_ids.index(plats_id)
        jap_idx = jap_ids.index(jap_id)
        gold_standard_indices.append((plats_idx, jap_idx))

# Contar cuántos pares del gold-standard superan el umbral de similitud de 0.75
umbral = 0.75
matches_above_threshold = 0

for plats_idx, jap_idx in gold_standard_indices:
    similarity = cosine_similarities[plats_idx, jap_idx].item()
    if similarity > umbral:
        matches_above_threshold += 1
        print(f"Gold-standard match: Plats ID {plats_ids[plats_idx]} - Jap ID {jap_ids[jap_idx]} con similitud {similarity:.4f}")

# Imprimir el número total de matches que superan el umbral
print(f"Número de matches del gold-standard que superan una similitud de {umbral}: {matches_above_threshold}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Gold-standard match: Plats ID plats_a_la_carta_C6528 - Jap ID gastronomia_japonesa_C10 con similitud 0.9665
Gold-standard match: Plats ID plats_a_la_carta_C6952 - Jap ID gastronomia_japonesa_C15 con similitud 0.7598
Número de matches del gold-standard que superan una similitud de 0.75: 2


In [ ]:
# Aplicar una máscara para encontrar todas las similitudes mayores al umbral
mask = cosine_similarities > umbral

# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Contador de matches totales que superan el umbral
total_matches_above_threshold = len(indices)

# Imprimir los pares que superan el umbral
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de 'plats' es similar a la línea {j} de 'jap' con similitud {similarity:.4f}")

# Mostrar el número total de matches que superan el umbral
print(f"Número total de matches que superan una similitud de {umbral}: {total_matches_above_threshold}")

Línea 51 de 'plats' es similar a la línea 8 de 'jap' con similitud 0.7852
Línea 51 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.7899
Línea 51 de 'plats' es similar a la línea 16 de 'jap' con similitud 0.7877
Línea 356 de 'plats' es similar a la línea 7 de 'jap' con similitud 0.7576
Línea 356 de 'plats' es similar a la línea 8 de 'jap' con similitud 0.7736
Línea 356 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.8682
Línea 356 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7707
Línea 425 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.7974
Línea 476 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.7700
Línea 577 de 'plats' es similar a la línea 7 de 'jap' con similitud 0.7917
Línea 577 de 'plats' es similar a la línea 8 de 'jap' con similitud 0.8298
Línea 577 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.9034
Línea 577 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.8263
Línea 577 de 'plats'

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Función para obtener solo la primera writtenRep
def get_first_written_rep(written_rep):
    return written_rep.split(",")[0].strip() if isinstance(written_rep, str) else ""

# Leer los archivos CSV
plats_df = pd.read_csv(plats, encoding='utf-8')
jap_df = pd.read_csv(jap, encoding='utf-8')
gold_standard_df = pd.read_csv(gold, encoding='utf-8')  # Cargar el gold-standard

# Limpiar el texto de las writtenRep eliminando "_[ja]"
plats_df['writtenRep'] = plats_df['writtenRep'].str.replace('_[ja]', '', regex=False)
jap_df['writtenRep'] = jap_df['writtenRep'].str.replace('_[ja]', '', regex=False)

# Concatenar 'prefLabel' y la primera 'writtenRep' de ambos archivos en una sola cadena para cada fila
plats_lines = plats_df['prefLabel'] + ". " + plats_df['writtenRep'].apply(get_first_written_rep)
jap_lines = jap_df['prefLabel'] + ". " + jap_df['writtenRep'].apply(get_first_written_rep)

# Limpiar y eliminar filas vacías
plats_lines = plats_lines.dropna().str.strip().tolist()
jap_lines = jap_lines.dropna().str.strip().tolist()

# Obtener los embeddings de las líneas de ambos archivos
plats_embeddings = model.encode(plats_lines, convert_to_tensor=True)
jap_embeddings = model.encode(jap_lines, convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Diccionario para convertir IDs a índices
plats_ids = plats_df['concepto'].tolist()
jap_ids = jap_df['concepto'].tolist()

# Leer los pares correctos del gold-standard
gold_standard_matches = set(zip(gold_standard_df['Plats_a_la_carta_ID'], gold_standard_df['Gastronomia_japonesa_ID']))

# Convertir el gold-standard a índices de filas y columnas de las matrices
gold_standard_indices = []
for plats_id, jap_id in gold_standard_matches:
    if plats_id in plats_ids and jap_id in jap_ids:
        plats_idx = plats_ids.index(plats_id)
        jap_idx = jap_ids.index(jap_id)
        gold_standard_indices.append((plats_idx, jap_idx))

# Contar cuántos pares del gold-standard superan el umbral de similitud de 0.75
umbral = 0.75
matches_above_threshold = 0

for plats_idx, jap_idx in gold_standard_indices:
    similarity = cosine_similarities[plats_idx, jap_idx].item()
    if similarity > umbral:
        matches_above_threshold += 1
        print(f"Gold-standard match: Plats ID {plats_ids[plats_idx]} - Jap ID {jap_ids[jap_idx]} con similitud {similarity:.4f}")

# Imprimir el número total de matches que superan el umbral
print(f"Número de matches del gold-standard que superan una similitud de {umbral}: {matches_above_threshold}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Número de matches del gold-standard que superan una similitud de 0.75: 0


In [ ]:
print(jap_lines)

['Gastronomia. dashi', 'Begudes. sake', 'Condiments. sal_de_sèsam', 'Gastronomia. sashimi', 'Gastronomia. shoyu', 'Gastronomia. sukiyaki', 'Gastronomia. sushi', 'Gastronomia. tempura', 'Gastronomia. teriyaki', 'Alimentació. wakame', 'Alimentació. xiitake', 'Gastronomia. katsuobushi', 'Gastronomia. yakitori', 'Alimentació. kombu', 'Alimentació. matsutake', 'Begudes. mirin', 'Alimentació. miso', 'Alimentació. mongeta_azuki', 'Alimentació. rave_japonès', 'Gastronomia. rotlle_de_sushi_[sushi:_ja]']


In [ ]:
# Aplicar una máscara para encontrar todas las similitudes mayores al umbral
mask = cosine_similarities > umbral

# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Contador de matches totales que superan el umbral
total_matches_above_threshold = len(indices)

# Imprimir los pares que superan el umbral
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de 'plats' es similar a la línea {j} de 'jap' con similitud {similarity:.4f}")

# Mostrar el número total de matches que superan el umbral
print(f"Número total de matches que superan una similitud de {umbral}: {total_matches_above_threshold}")

Línea 63 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7672
Línea 223 de 'plats' es similar a la línea 1 de 'jap' con similitud 0.7520
Línea 223 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7711
Línea 348 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7548
Línea 424 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7582
Línea 525 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7674
Línea 571 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7548
Línea 594 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7810
Línea 619 de 'plats' es similar a la línea 9 de 'jap' con similitud 0.7665
Línea 619 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7772
Línea 665 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7746
Línea 782 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7617
Línea 869 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7555
Línea 1127 de '

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Función para obtener solo la primera writtenRep
def get_first_writtenrep(row, writtenrep_col):
    return row[writtenrep_col].split(",")[0].strip() if isinstance(row[writtenrep_col], str) else ""

# Función para crear la oración con contexto forzado
def create_sentence(row, writtenrep_col):
    first_writtenrep = get_first_writtenrep(row, writtenrep_col)
    return f"{first_writtenrep} és un plat" if first_writtenrep else None

# Leer los archivos CSV
plats_df = pd.read_csv(plats, encoding='utf-8')
jap_df = pd.read_csv(jap, encoding='utf-8')
gold_standard_df = pd.read_csv(gold, encoding='utf-8')  # Cargar el gold-standard

# Limpiar el texto de las writtenRep eliminando "_[ja]"
plats_df['writtenRep'] = plats_df['writtenRep'].str.replace('_[ja]', '', regex=False)
jap_df['writtenRep'] = jap_df['writtenRep'].str.replace('_[ja]', '', regex=False)

# Crear las oraciones con el contexto "és un plat" usando la primera writtenRep
plats_lines = plats_df.apply(lambda row: create_sentence(row, 'writtenRep'), axis=1).dropna().tolist()
jap_lines = jap_df.apply(lambda row: create_sentence(row, 'writtenRep'), axis=1).dropna().tolist()

# Obtener los embeddings de las líneas de ambos archivos
plats_embeddings = model.encode(plats_lines, convert_to_tensor=True)
jap_embeddings = model.encode(jap_lines, convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Diccionario para convertir IDs a índices
plats_ids = plats_df['concepto'].tolist()
jap_ids = jap_df['concepto'].tolist()

# Leer los pares correctos del gold-standard
gold_standard_matches = set(zip(gold_standard_df['Plats_a_la_carta_ID'], gold_standard_df['Gastronomia_japonesa_ID']))

# Convertir el gold-standard a índices de filas y columnas de las matrices
gold_standard_indices = []
for plats_id, jap_id in gold_standard_matches:
    if plats_id in plats_ids and jap_id in jap_ids:
        plats_idx = plats_ids.index(plats_id)
        jap_idx = jap_ids.index(jap_id)
        gold_standard_indices.append((plats_idx, jap_idx))

# Contar cuántos pares del gold-standard superan el umbral de similitud de 0.75
umbral = 0.75
matches_above_threshold = 0

for plats_idx, jap_idx in gold_standard_indices:
    similarity = cosine_similarities[plats_idx, jap_idx].item()
    if similarity > umbral:
        matches_above_threshold += 1
        print(f"Gold-standard match: Plats ID {plats_ids[plats_idx]} - Jap ID {jap_ids[jap_idx]} con similitud {similarity:.4f}")

# Imprimir el número total de matches que superan el umbral
print(f"Número de matches del gold-standard que superan una similitud de {umbral}: {matches_above_threshold}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Gold-standard match: Plats ID plats_a_la_carta_C7091 - Jap ID gastronomia_japonesa_C16 con similitud 0.8802
Gold-standard match: Plats ID plats_a_la_carta_C6911 - Jap ID gastronomia_japonesa_C14 con similitud 0.8646
Gold-standard match: Plats ID plats_a_la_carta_C6528 - Jap ID gastronomia_japonesa_C10 con similitud 1.0000
Gold-standard match: Plats ID plats_a_la_carta_C2926 - Jap ID gastronomia_japonesa_C1 con similitud 0.8385
Gold-standard match: Plats ID plats_a_la_carta_C6952 - Jap ID gastronomia_japonesa_C15 con similitud 0.8916
Gold-standard match: Plats ID plats_a_la_carta_C6648 - Jap ID gastronomia_japonesa_C12 con similitud 0.8595
Número de matches del gold-standard que superan una similitud de 0.75: 6


In [ ]:
# Aplicar una máscara para encontrar todas las similitudes mayores al umbral
mask = cosine_similarities > umbral

# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Contador de matches totales que superan el umbral
total_matches_above_threshold = len(indices)

# Imprimir los pares que superan el umbral
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de 'plats' es similar a la línea {j} de 'jap' con similitud {similarity:.4f}")

# Mostrar el número total de matches que superan el umbral
print(f"Número total de matches que superan una similitud de {umbral}: {total_matches_above_threshold}")

Streaming output truncated to the last 5000 lines.
Línea 3249 de 'plats' es similar a la línea 14 de 'jap' con similitud 0.8210
Línea 3249 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.8466
Línea 3249 de 'plats' es similar a la línea 16 de 'jap' con similitud 0.9133
Línea 3250 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.8470
Línea 3250 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.7611
Línea 3250 de 'plats' es similar a la línea 16 de 'jap' con similitud 0.8140
Línea 3251 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7694
Línea 3252 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7557
Línea 3253 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7906
Línea 3280 de 'plats' es similar a la línea 13 de 'jap' con similitud 0.7661
Línea 3281 de 'plats' es similar a la línea 0 de 'jap' con similitud 0.8029
Línea 3281 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.7926
Línea 3281 de 'plats' es s

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Leer las líneas de ambos archivos
with open(plats, 'r', encoding='utf-8') as f1, open(jap, 'r', encoding='utf-8') as f2:
    plats_lines = f1.readlines()
    jap_lines = f2.readlines()

# Limpiar y eliminar saltos de línea
plats_lines = [line.strip() for line in plats_lines if line.strip()]
jap_lines = [line.strip() for line in jap_lines if line.strip()]

# Obtener los embeddings de las líneas de ambos archivos
plats_embeddings = model.encode(plats_lines, convert_to_tensor=True)
jap_embeddings = model.encode(jap_lines, convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.9

# Aplicar una máscara para encontrar las similitudes mayores al umbral
mask = cosine_similarities > umbral

# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Imprimir los pares de líneas con similitud mayor al umbral
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de 'plats' es similar a la línea {j} de 'jap' con similitud {similarity:.4f}")
    print(f"Plats: {plats_lines[i]}")
    print(f"Jap: {jap_lines[j]}\n")


Línea 4497 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.9634
Plats: mirin Mirin mirin mirin mirin mirin
Jap: mirin mirin mirin mirin

Línea 6146 de 'plats' es similar a la línea 1 de 'jap' con similitud 0.9281
Plats: sake Sake sake sake saké sakè
Jap: sake sake saké saki sake saké saki



In [ ]:
import re
import string
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

# Función de preprocesamiento
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar puntuación
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Eliminar espacios en blanco innecesarios
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Leer las líneas de ambos archivos
with open(plats, 'r', encoding='utf-8') as f1, open(jap, 'r', encoding='utf-8') as f2:
    plats_lines = f1.readlines()
    jap_lines = f2.readlines()

# Preprocesar las líneas
plats_lines = [preprocess_text(line) for line in plats_lines if line.strip()]
jap_lines = [preprocess_text(line) for line in jap_lines if line.strip()]

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Obtener los embeddings de las líneas preprocesadas de ambos archivos
plats_embeddings = model.encode(plats_lines, convert_to_tensor=True)
jap_embeddings = model.encode(jap_lines, convert_to_tensor=True)

# Normalizar los embeddings para similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.79

# Aplicar una máscara para encontrar las similitudes mayores al umbral
mask = cosine_similarities > umbral

# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Imprimir los pares de líneas con similitud mayor al umbral
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de 'plats' es similar a la línea {j} de 'jap' con similitud {similarity:.4f}")
    print(f"Plats: {plats_lines[i]}")
    print(f"Jap: {jap_lines[j]}\n")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Línea 356 de 'plats' es similar a la línea 8 de 'jap' con similitud 0.8282
Plats: bergkäse bergkäse bergkäse bergkäse bergkäse bergkäse
Jap: teriyaki teriyaki teriyaki teriyaki

Línea 356 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.8386
Plats: bergkäse bergkäse bergkäse bergkäse bergkäse bergkäse
Jap: yakitori yakitori yakitori yakitori

Línea 356 de 'plats' es similar a la línea 16 de 'jap' con similitud 0.8000
Plats: bergkäse bergkäse bergkäse bergkäse bergkäse bergkäse
Jap: miso miso miso miso

Línea 577 de 'plats' es similar a la línea 8 de 'jap' con similitud 0.8994
Plats: brioix brioche brioche brioche brioche brioche
Jap: teriyaki teriyaki teriyaki teriyaki

Línea 577 de 'plats' es similar a la línea 12 de 'jap' con similitud 0.8646
Plats: brioix brioche brioche brioche brioche brioche
Jap: yakitori yakitori yakitori yakitori

Línea 577 de 'plats' es similar a la línea 15 de 'jap' con similitud 0.8603
Plats: brioix brioche brioche brioche brioche brioche
Jap: mi

si que tiene sentido la similitud de embeddings porque se han conectado aquellos que tienen las traducciones más similares, que son extranjerismos para otros idiomas.